In [ ]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard

In [ ]:
lr = 0.0005
batch_size = 64
image_size = 64
channels_img = 3
channels_noise = 256


# For how many channels Generator and Discriminator should use
features_d = 16
features_g = 16

my_transforms = transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)


In [ ]:
dataset= '/content/pokemon'

In [ ]:
folder = datasets.ImageFolder(dataset,transform=my_transforms)

In [ ]:
dataloader = DataLoader(folder,batch_size=batch_size,shuffle = True)

In [ ]:

class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            # N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # N x features_d x 32 x 32
            nn.Conv2d(features_d, features_d * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2),
            nn.Conv2d(
                features_d * 2, features_d * 4, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2),
            nn.Conv2d(
                features_d * 4, features_d * 8, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2),
            # N x features_d*8 x 4 x 4
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
            # N x 1 x 1 x 1
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()

        self.net = nn.Sequential(
            # N x channels_noise x 1 x 1
            nn.ConvTranspose2d(
                channels_noise, features_g * 16, kernel_size=4, stride=1, padding=0
            ),
            nn.BatchNorm2d(features_g * 16),
            nn.ReLU(),
            # N x features_g*16 x 4 x 4
            nn.ConvTranspose2d(
                features_g * 16, features_g * 8, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(),
            nn.ConvTranspose2d(
                features_g * 8, features_g * 4, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(
                features_g * 4, features_g * 2, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(),
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
netD = Discriminator(channels_img,features_d).to(device)
netG= Generator(channels_noise,channels_img,features_g).to(device)

In [ ]:
optimizerD = optim.Adam(netD.parameters(),lr = lr, betas=(0.5,0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

In [ ]:
netG.train()
netD.train()

Discriminator(
  (net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Conv2d(128, 1, kernel_size=(4, 4), stride=(2, 2))
    (12): Sigmoid()
  )
)

In [ ]:
criterion =  nn.BCELoss()

In [ ]:
real_label  = 1
fake_label = 0

In [ ]:
fixed_noise = torch.randn(64,channels_noise,1,1).to(device)
writer_real = SummaryWriter(f"runs/GAN_MNIST/test_real")
writer_fake = SummaryWriter(f"runs/GAN_MNIST/test_fake")
step = 0

In [ ]:
num_epochs= 600

In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (data,targets) in enumerate(dataloader):
    data = data.to(device)
    batch_size = data.shape[0]


    netD.zero_grad()
    label = (torch.ones(batch_size) * 0.9).to(device)
    output = netD(data).reshape(-1)
    lossD_real = criterion(output,label)
    D_x= output.mean().item()

    noise = torch.randn(batch_size,channels_noise,1,1).to(device)
    fake = netG(noise)
    label = (torch.ones(batch_size) * 0.1).to(device)
    

    output = netD(fake.detach()).reshape(-1)
    lossD_fake = criterion(output, label)

    lossD = lossD_real + lossD_fake
    lossD.backward()
    optimizerD.step()

        ### Train Generator: max log(D(G(z)))
    netG.zero_grad()
    label = torch.ones(batch_size).to(device)
    output = netD(fake).reshape(-1)
    lossG = criterion(output, label)
    lossG.backward()
    optimizerG.step()

        # Print losses ocassionally and print to tensorboard
    if batch_idx % 100 == 0: 


      step += 1
      print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {lossD:.4f}, loss G: {lossG:.4f} D(x): {D_x:.4f}"
            )

      with torch.no_grad():

        fake = netG(fixed_noise)
        img_grid_real = torchvision.utils.make_grid(data[:32], normalize=True)
        img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
        writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
        writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )

      


Epoch [0/600] Batch 0/13                   Loss D: 1.2839, loss G: 1.3207 D(x): 0.4961
Epoch [1/600] Batch 0/13                   Loss D: 0.9572, loss G: 3.7662 D(x): 0.7518
Epoch [2/600] Batch 0/13                   Loss D: 0.7872, loss G: 3.6918 D(x): 0.7202
Epoch [3/600] Batch 0/13                   Loss D: 1.2020, loss G: 5.3721 D(x): 0.8510
Epoch [4/600] Batch 0/13                   Loss D: 0.8653, loss G: 3.6022 D(x): 0.6555
Epoch [5/600] Batch 0/13                   Loss D: 0.8612, loss G: 2.9853 D(x): 0.6385
Epoch [6/600] Batch 0/13                   Loss D: 0.7717, loss G: 3.9708 D(x): 0.8012
Epoch [7/600] Batch 0/13                   Loss D: 0.7772, loss G: 3.4170 D(x): 0.9132
Epoch [8/600] Batch 0/13                   Loss D: 1.2209, loss G: 2.1192 D(x): 0.6504
Epoch [9/600] Batch 0/13                   Loss D: 1.0990, loss G: 1.6471 D(x): 0.5632
Epoch [10/600] Batch 0/13                   Loss D: 1.1145, loss G: 1.2942 D(x): 0.5340
Epoch [11/600] Batch 0/13                 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/runs/GAN_MNIST                                                                                                        

Reusing TensorBoard on port 6006 (pid 666), started 1:59:23 ago. (Use '!kill 666' to kill it.)

<IPython.core.display.Javascript object>